# Functional Programming

The style of programming we've been doing is called **imperative** or **procedural**.  Statements run in sequence and change a program's state.

**state** can be thought of as the status of all variables at a given time. Imperative programming relies heavily on functions updating state.

As we said early on, Python is multi-paradigm. 

> "[...] practicality beats purity."
> 
> - The Zen of Python

Languages like LISP, Haskell, and Racket are purely functional & differ significantly from procedural & object-oriented languages.

Functional programming uses a definition of functions more compatible with the mathematical definition. Instead of the recipe model of procedural programming, mathematical functions take input(s) and return an output. 

These functions do not have the concept of "state", that is, calling a function in math creates a mapping from inputs to outputs.

When we call `sin(x)` we do not speak of it modifying its inputs, just returning a value.

Similarly, when we workin a functional style we'll often write smaller functions that we chain together, instead of long procedures that rely on internal state.

Python has many features that stem from pure functional languages & support functional programming:

- Functions as first class objects
- Lambda expressions
- map/filter
- `functools`
- comprehensions

## Functions are "first-class objects"

A key feature of Python that makes it possible to write code in the functional style is the fact that functions are objects. (Everything in Python is an object.)

This means functions don't have special rules about how they can be used, any variable can reference a function. (Remember, a variable is an association between a name & object.)

In [1]:
def echo(message):
    print(message)
    print(message)
    
print(f"echo = {echo}")
print(f"type(echo) = {type(echo)}")

echo = <function echo at 0x10760c820>
type(echo) = <class 'function'>


In [2]:
# we can assign other names to objects, including functions

x = echo

x("hello")

hello
hello


In [3]:
id(x), id(echo)

(4418750496, 4418750496)

In [4]:
# we can also store functions in other types

func_list = [print, echo, print, echo]
for i, func in enumerate(func_list):
    func(i)

0
1
1
2
3
3


In [5]:
# dictionaries too
func_mapping = {False: print, True: echo}

print_twice = True
func_mapping[True]("twice")

print_twice = False
func_mapping[print_twice]("once")

twice
twice
once


In [20]:
# we can pass functions into other functions

def add(a, b):
    return a + b

def sub(a, b):
    return a - b

def perform_op(op_func, a, b):
    return op_func(a, b)

print("add, 3, 4 = ", perform_op(add, 3, 4))
print("sub, 3, 4 = ", perform_op(sub, 3, 4))

add, 3, 4 =  7
sub, 3, 4 =  -1


In [ ]:
# and we can return functions from other functions

def get_op(name):
    if name == "div":
        def f(a, b):
            return a / b
    elif name == "mod":
        def f(a, b):
            return a % b
    return f

In [ ]:
fn = get_op("mod")
perform_op(fn, 10, 3)

In [17]:
x = [("Nick", 1), ("Yusong", 9000), ("Emma", 100)]

def negate(y):
    return -y

def second_key(item):
    return item[1]


In [18]:
x.sort(key=second_key)
print(x)

[('Nick', 1), ('Emma', 100), ('Yusong', 9000)]


In [14]:
second_key(x[0])

1

In [9]:
x.sort()
print(x)

[('Emma', 100), ('Nick', 1), ('Yusong', 9)]


In [ ]:
def second_key(item):
    return item[1]
    
x.sort(key=lambda item: item[1])
print(x)

#x.sort(key=negate)
#print(x)

## lambda functions

Python also provides another way to generate function objects.

These are called lambda functions (aka anonymous functions), which:

- Are expressions that return a function object that can be called later without providing a name (hence ``anonymous")
- Can be used in places where def statement is not syntactically legal (inside a literal list, inlined as a function argument, etc.)

The body of an lambda function is a single expression, not a block of statements.  The body is similar to a return statement in a def statement.

```python

lambda arg1, arg2: expression

# essentially the same as

def __(arg1, arg2):
    return expression
```

(0 or more arguments, but *must* have an expression)

## Reminder: expressions vs. statements

Everything in Python is either an expression or a statement. 

An expression evaluates to a value, examples include:

* `42`
* `"hello world"`
* `10 * 5`
* `f(1, 2, 3)`
* `[1, 2, 3]`
* `l[0]` 
* `lambda arg1, arg2: arg1 + arg2`

Notice that all of these could be found on the right hand side of an assignment (e.g. `x = 10 * 5`)

Expresssions are valid in assignment, function calls, sequence values, etc.  (Anywhere a value is needed.)

```
# in assignment
x = 42
x = 10 * 5
x = [1, 2, 3]

# in function calls
f(42)
f(10 * 5)
f([1, 2, 3])

# in complex types
[42, [1, 2, 3], lambda x: x**2]
{10*5: f(10, 5)}
```

To contrast, statements perform an action.

* `x = 1`
* `if x: ...`
* `def f(a): ...` 
* `import math`

They are prohibted where types are required:

```
 # not allowed
x = if y > 0: 
   7

z = def f(a): 
   ...
```

A statement will often have multiple expressions within it. Many statements (but not all) use indented blocks.

When it comes to `lambda`:
* a `lambda` defines a function that maps input to a single expression, `def` can be used if more was needed
* a `lambda` is itself an expression, it can be used anywhere other expresssions are needed

In [21]:
# can fit places a function definition can't
# such as being used as a parameter
perform_op(lambda a, b: a * b, 5, 6)

30

In [ ]:
lambda s: s.upper()

In [23]:
sorted(["abc", "Abc", "ABC", "AbC"], key=lambda s: s.upper())

['abc', 'Abc', 'ABC', 'AbC']

In [24]:
# can be assigned to a variable
mul = lambda a, b: a * b
mul(5, 6)

# same as
def mul2(a, b):
    return a * b

In [25]:
type(mul), type(mul2)

(function, function)

General rule: If you're giving a lambda a name, use a function.

In [ ]:
# common use case
names = ["adam", "Ziwe", "Bo", "JENNY"]
names.sort()
print(names)  # case sensitive, lower-case a comes after Z

In [ ]:
names.sort(key = lambda x: x.upper())
print(names)

## Functional Methods

Python also has several built in methods that are useful when writing programs with a functional mindset.

`map`, `filter`, `functools`

#### `map(function, iterable1, [...iterableN])`

Returns a new iterable that calls `function` with parameters from `iterable1 ... iterableN`.

In [1]:
def add_two(x):
    return x + 2

for x in map(add_two, [1, 2, 3]):
    print(x)

3
4
5


In [2]:
help(map)

Help on class map in module builtins:

class map(object)
 |  map(func, *iterables) --> map object
 |  
 |  Make an iterator that computes the function using arguments from
 |  each of the iterables.  Stops when the shortest iterable is exhausted.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



In [3]:
x = list(map(add_two, [1, 2, 3]))
print(x)

[3, 4, 5]


In [11]:
# commonly used with lambdas
for x in map(lambda x, y: x+y, ("A", "B", "C"), ["!", "?", "."]):
    print(x)

A!
B?
C.


In [12]:
# number of parameters must match number of iterables
for x in map(lambda x, y, z: x+y*z, ("A", "B", "C"), ["!", "?", "."], [2, 3, 4]):
    print(x)

A!!
B???
C....


In [14]:
# operator module contains all of the common operators in function form
import operator
operator.sub(20, 5)

15

In [15]:

list(map(operator.sub, [20, 19], [10, 9]))

[10, 10]

In [20]:
# the result of `map` is a new kind of iterable (in fact a generator, which we'll cover next)

# possible to pass into set or list 
#  or use anywhere you can use an iterable
set(map(lambda x: x * 3, ("A", "B", "C")))

{'AAA', 'BBB', 'CCC'}

#### `filter(function, iterable)` 

returns an iterable that contains all items from iterable for which `function(item)` returns True

In [21]:
list(map(lambda s: s.title(), filter(str.isupper, ["a", "ABC", "AbCdeF", "XYZ"])))

['Abc', 'Xyz']

In [22]:
list(filter(lambda s: s.isupper(), ["a", "ABC", "AbCdeF", "XYZ"]))

['ABC', 'XYZ']

In [23]:
list(map(lambda s: s.lower(), filter(lambda s: s.isupper(), ["a", "ABC", "AbCdeF", "XYZ"])))

['abc', 'xyz']

In [24]:
g = (x**2 for x in filter(lambda x: x % 2 != 0, range(20)))
list(g)

[1, 9, 25, 49, 81, 121, 169, 225, 289, 361]

#### functools

In [ ]:
import functools
[name for name in dir(functools) if name[0].islower()]

``functools.reduce(function, iterable[, initializer])``

Apply ``function`` to pairs of items successively and return a single value as the result. You can optionally specify the initial value.


In [ ]:
import functools 
import operator 

#accumulator = 0
#for item in my_list:
#    accumulator += item

# 1st iteration: Call operator.add(1,2) -> 3 
# 2nd iteration: Call operator.add(3,3) -> 6 
# 3rd iteration: Call operator.add(6,4) -> 10 
# final result = 10 
functools.reduce(operator.add, [1,2,3,4])

In [ ]:
names = ["Ben", "Martha", "Susan"]
# 1st iteration: call f(0, "Ben") -> 0 + len("Ben") -> 3
# 2nd iteration: call f(3, "Martha") -> 3 + len("Martha") -> 9
# 3rd iteration: call f(9, "Susan") -> 9 + len("Susan") -> 14
functools.reduce(lambda a, b: a + len(b), names, 0)

In [ ]:
# What happens if you pass in an initial value 
# 1st iteration: Call operator.mul(2,1) -> 2 
# 2nd iteration: Call operator.mul(2,2) -> 4 
# 3rd iteration: Call operator.mul(4,3) -> 12 
# 4th iteration: Call operator.mul(12,4) -> 48 
# Final result = 48 
functools.reduce(operator.mul, [1,2,3,4], 0)

In [ ]:
functools.reduce(lambda a,b: a+b, [], 0)

```functools.partial(func, *args, **kwargs)```

`functools.partial` returns a new function that "binds" any passed args & kwargs, and leaves other parameters unbound.

In [25]:
import operator
operator.mul(2, 10)

20

In [26]:
import functools
negate = functools.partial(operator.mul, -1)
negate(5)

-5

In [27]:
def calls_twice(f):
    print(f())
    print(f())
    

g = functools.partial(operator.mul, 4, 4)
#print(g())
calls_twice(g)



16
16


In [28]:
print_ex = functools.partial(print, "x", sep="!")
print_ex("a", "b", "c")
print("x", "a", "b", "c", sep="!")

x!a!b!c
x!a!b!c


In [32]:
# parameters must be valid
print_foo = functools.partial(print, foo="x")

In [30]:
print_foo("hello")

TypeError: 'foo' is an invalid keyword argument for print()

In [34]:
# another way to deal with functions we're calling with the same args repeatedly
def request_page(url, verify, cache=True, send_cookies=False, https_only=True):
    pass

secure_request = functools.partial(request_page, verify=True, https_only=True)

In [38]:
secure_request("", verify=False)

## List Comprehensions

Generate a new list from an existing iterable.

Same syntax as generator expression but inside `[]`:

```python
new_list = [expression for var in iterable]

# or 

new_list = [expression for var in iterable if condition]
```

In [ ]:
powers_of_two = [2 ** n for n in range(10)]
print(powers_of_two)

In [ ]:
# possible to nest comprehensions, but beware readability
faces = ("K", "Q", "J")
suits = ("♠", "♣", "♦", "♥")
cards = [(face + suit) for face in faces for suit in suits if face != "K"]
print(cards)

## Set & Dict Comprehensions

Also possible to make `set` and `dict` comprehensions by using `{}`.

In [ ]:
powers_of_two_set = {2 ** n for n in [1,1,2,2,3,3,3,4,4,4]}
print(powers_of_two_set)

In [ ]:

powers_of_two_mapping = {n: f(n) for n in range(5) if n > 0}
print(powers_of_two_mapping)